# Configure AI Core

Let's first connect to the AI Core client using credentials stored in `.env`.

In [13]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [8]:
import os
# Load Library
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection
ai_core_client = AICoreV2Client(
    base_url = os.environ["AICORE_BASE_URL"],
    auth_url=  os.environ["AICORE_AUTH_URL"], 
    client_id = os.environ["AICORE_CLIENT_ID"],
    client_secret = os.environ["AICORE_CLIENT_SECRET"],
    resource_group=os.environ['AICORE_RESOURCE_GROUP']
)

Now, let's store Docker credentials as a secret in AI Core.

In [12]:
response = ai_core_client.docker_registry_secrets.create(
    name = "docker-secret-aman",
    data = {
        ".dockerconfigjson": f'{{\"auths\":{{\"docker.io/amanichopra\":{{\"username\":\"amanichopra\",\"password\":\"{os.environ["DOCKER_PAT"]}\"}}}}}}'
    }
)
print(response.__dict__)

{'message': 'secret has been created'}


Now, let's onboard a GitHub repo onto AI Core.

In [ ]:
response = ai_core_client.repositories.create(
    name = "llm-serving-benchmarking-gh-repo",
    url = "https://github.com/amanichopra/llm-serving-framework-benchmarking.git",
    username = "amanichopra",
    password = os.environ['GH_PAT']
)
print(response.__dict__)

# Ollama

Let's create an AI Core application for to serve via Ollama.

In [ ]:
response = ai_core_client.applications.create(
    application_name = "ollama",
    repository_url = "https://github.com/amanichopra/llm-serving-framework-benchmarking.git",
    path = "./ollama",
    revision = "HEAD"
)
print(response.__dict__)

List all applications to ensure the new app has been created.

In [19]:
response = ai_core_client.applications.query()

for app in response.resources:
    print(app.__dict__)

{'path': 'pipelines', 'revision': 'HEAD', 'repository_url': 'https://github.tools.sap/D070429/ai_core_information_extraction', 'application_name': 'ai-core-information-extraction'}
{'path': 'LearningScenarios', 'revision': 'HEAD', 'repository_url': 'https://github.tools.sap/I589259/aicore-pipelines', 'application_name': 'andreas-learning-scenarios-app'}
{'path': 'LearningScenarios', 'revision': 'HEAD', 'repository_url': 'https://github.tools.sap/D070429/aicore-pipelines', 'application_name': 'christians-learning-scenarios-app'}
{'path': 'LearningScenarios', 'revision': 'HEAD', 'repository_url': 'https://github.tools.sap/I522484/aicore-pipelines', 'application_name': 'christoph-learning'}
{'path': 'templates', 'revision': 'HEAD', 'repository_url': 'https://github.com/fyx99/CI-CD-with-AI-Core#main', 'application_name': 'cicd-felix-application'}
{'path': 'templates', 'revision': 'HEAD', 'repository_url': 'https://github.com/fyx99/Cloud-Connector-with-AI-Core', 'application_name': 'cloud-c

Ensure the application is synced to the GitHub workflow template.

In [20]:
print(ai_core_client.applications.refresh(application_name='ollama').message)
# Get application status
response = ai_core_client.applications.get_status(application_name='ollama')
print(response.message)

for sync_status in response.sync_ressources_status:
    print(sync_status.__dict__)

A refresh of the application has been scheduled.
Failed to load target state: failed to generate manifest for source 1 of 1: rpc error: code = Unknown desc = failed to list refs: remote repository is empty
